<a href="https://colab.research.google.com/github/albisbub/RFIC-Pitches/blob/master/_notebooks%5C2020-04-29-ALK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RFIC MEETING 7
> Alaska Airlines Buy Pitch

- toc:true
- branch: master
- badges: true
- comments: true
- author: Blaise and Abby
- permalink: /ALK/
- categories: [fastpages, jupyter]

In [303]:
#hide
#Requirements:
!pip install requests bs4 pandas yahoofinancials yahoo_fin requests_html

In [0]:
#hide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from datetime import timedelta, datetime, date
%config InlineBackend.figure_format = 'retina'

chart_width = 550
chart_height= 400

import gspread
from oauth2client.service_account import ServiceAccountCredentials

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import requests
# !pip install 
from bs4 import BeautifulSoup
import pandas as pd
from yahoofinancials import YahooFinancials
from yahoo_fin.stock_info import get_quote_table, get_stats, get_live_price, get_income_statement, get_balance_sheet, get_cash_flow
# !pip install dict
# from dict import dict
def getQuote(ticker):
    return str(round(get_live_price(ticker), 2))

def finviz_summaryTable(symbols):
    # Get Column Header
    req = requests.get("https://finviz.com/quote.ashx?t=FB")
    soup = BeautifulSoup(req.content, 'html.parser')
    table = soup.find_all(lambda tag: tag.name=='table')
    rows = table[8].findAll(lambda tag: tag.name=='tr')
    out=[]
    for i in range(len(rows)):
        td=rows[i].find_all('td')
        out=out+[x.text for x in td]

    ls=['Ticker','Sector','Sub-Sector','Country']+out[::2]

    dict_ls={k:ls[k] for k in range(len(ls))}
    df=pd.DataFrame()
    print('Fetching Data...')
    for j in range(len(symbols)):

      try:

        req = requests.get("https://finviz.com/quote.ashx?t="+symbols[j])
        if req.status_code !=200:
            continue
        soup = BeautifulSoup(req.content, 'html.parser')
        table = soup.find_all(lambda tag: tag.name=='table')
        rows=table[6].findAll(lambda tag: tag.name=='tr')
        sector=[]
        for i in range(len(rows)):
            td=rows[i].find_all('td')
            sector=sector+[x.text for x in td]
        sector=sector[2].split('|')
        rows = table[8].findAll(lambda tag: tag.name=='tr')
        out=[]
        for i in range(len(rows)):
            td=rows[i].find_all('td')
            out=out+[x.text for x in td]
        out=[symbols[j]]+sector+out[1::2]
        out_df=pd.DataFrame(out).transpose()
        df=df.append(out_df,ignore_index=True)
        print('Done!')
        
        df=df.rename(columns=dict_ls)   

      except BaseException as E:
        print("Could not find data:", E)

    return(df)
    
def yf_summaryTable(tickerList):
    values = []
    for i in tickerList:
      try:
        value = get_quote_table(i , dict_result = True)
        values.append(value)
      except BaseException as e:
        print(e)
             
    outD = dict(zip(tickerList,values))
            
    return outD
        
def yf_stats_compTable(tickerList):
    values = []
    for i in tickerList:
      try:
        value = get_stats(i)
        values.append(value)
      except BaseException as e:
        print(e)
             
    outD= dict(zip(tickerList,values))
            
    return outD
               
def get_prices(tickerlist, start, end, period):
    yahoo_financials = YahooFinancials(tickerlist)
    jsonDATA = yahoo_financials.get_historical_price_data(start,end, period)
    return jsonDATA

In [0]:
#hide
def make_multi_LineGraph(data, logged):
  df = data
  if logged == True:
    df['log Value'] =  np.around(np.log(df.Value), 4)
    aY = ('log Value:Q')
    scc = alt.Scale(type='log')
    color1 = alt.Color('Metric:N', legend=alt.Legend(title="Natural Log of Metrics", labelFontSize=15, titleFontSize=17),
                     scale=alt.Scale(scheme='tableau20'))
  else:
    aY = ('Value:Q')
    scc=alt.Scale(type='linear')
    color1 = alt.Color('Metric:N', legend=alt.Legend(title="Metrics",labelFontSize=15, titleFontSize=17),
                     scale=alt.Scale(scheme='tableau20'))   
  label = alt.selection_multi(fields=['Metric'], bind='legend', 
      on='mouseover',  # select on mouseover events
      nearest=True,    # select data point nearest the cursor
      empty='none'     # empty selection includes no data points
  )
  # define our base line chart of stock prices
  base = alt.Chart().mark_line().encode(
      alt.X('Year:T'),
      alt.Y(aY, scale=scc) ,
      color = color1 
      
      
      # alt.Color('Metric:N'), opacity = alt.condition(label, alt.value(1), alt.value(0.5))

  )

  graph = alt.layer(
      base, # base line chart
      
      # add a rule mark to serve as a guide line
      alt.Chart().mark_rule(color='#aaa').encode(
          x='Year:T'
      ).transform_filter(label),
      
      # add circle marks for selected time points, hide unselected points
      base.mark_circle().encode(
          opacity=alt.condition(label, alt.value(1), alt.value(0))
      ).add_selection(label),

      # add white stroked text to provide a legible background for labels
      base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
          text=aY
      ).transform_filter(label),

      # add text labels for stock prices
      base.mark_text(align='left', dx=5, dy=-5).encode(
          text=aY
      ).transform_filter(label),
      
      data=df
  ).properties(
      width=700,
      height=400
  )
  
  return graph


def formatPeers(inputs):
  ColumnList = list(inputs.columns)
  yearList = list(inputs.Ticker)

  print(ColumnList)
  print(yearList)


  transposed = inputs.transpose()
  cleaned = transposed.reset_index()

  new_header = cleaned.iloc[0] #grab the first row for the header
  cleaned = cleaned[1:] #take the data less the header row
  cleaned.columns = new_header #set the header row as the df header

  byYear = cleaned.set_index("Ticker").stack()

  df = pd.DataFrame(byYear, columns=['Value'])
  df.reset_index(inplace=True)
  print(df.columns)
  df.rename(columns={0:'Ticker'}, inplace=True)
  df.dropna(inplace=True)
  df['Value'] = pd.to_numeric(df['Value'], errors='coerce')

  new_header = df.iloc[0] #grab the first row for the header
  df = df[1:] #take the data less the header row
  df.columns = new_header #set the header row as the df header


  return df

def get_financials(tickerList):
  """
  list of lists
  same order as ticker list
  each list contains a list of datframes in the order income statement , balance sheet, cash flow
  """

  tickersDF = []
  for i in tickerList:

    income_statement = get_income_statement(i)
    
    balance_sheet = get_balance_sheet(i)
    
    cash_flow = get_cash_flow(i)

    finList = [income_statement, balance_sheet, cash_flow]
    tickersDF.append(finList)

  return tickersDF

def interactive_bat(data):


  source = data
  brush = alt.selection(type='interval', encodings=['x'])

  bars = alt.Chart().mark_bar().encode(
      x='Ticker:N',
      y='mean(Value):Q',
      opacity=alt.condition(brush, alt.OpacityValue(1), alt.OpacityValue(0.7)),
  ).add_selection(
      brush
  )

  line = alt.Chart().mark_rule(color='firebrick').encode(
      y='mean(Value):Q',
      size=alt.SizeValue(3)
  ).transform_filter(
      brush
  )

  graph = alt.layer(bars, line, data=source)

  return graph

  

In [0]:
#hide
subjectCompany = "ALK"
peers = ["AAL", "DAL", 'LUV', 'UAL']
tickerList = []
tickerList.append(subjectCompany)
for i in peers:

  tickerList.append(i)

In [0]:
#hide
all_financials_querry = get_financials(tickerList)
#Works
subjectTfinancials = all_financials_querry[0]

In [308]:
#hide
## Clean some shit up for charting
from datetime import date
today = date.today()

now = str(today.strftime("%m/%d/%Y"))
print(now)

for i in subjectTfinancials:
  a = i
  a.rename({'ttm': now}, inplace=True, axis = "columns")
  a.rename({'Breakdown': "Year"}, inplace=True, axis = "columns")


04/30/2020


Note: The most recent annual column is trailing twelve months data (often denoted TTM).

In [0]:
#hide
# !pip install pyfinmod
# from pyfinmod.financials import Financials
# parser = Financials(subjectCompany)

# (parser.cash_flow_statement)
# (parser.balance_sheet_statement)
# (parser.income_statement)

# firstDiff = True

# if firstDiff:
#   cash = (parser.cash_flow_statement).transpose().iloc[::-1].pct_change().reset_index().rename(columns={'index':'Year'})
#   balance = (parser.balance_sheet_statement).transpose().iloc[::-1].pct_change().reset_index().rename(columns={'index':'Year'})
#   income = (parser.income_statement).transpose().iloc[::-1].pct_change().reset_index().rename(columns={'index':'Year'})# income = subjectTfinancials[0]

income = subjectTfinancials[0]
balance = subjectTfinancials[1]
cash = subjectTfinancials[2]

# Subjet Company Financials

## Income Statement

In [310]:
#collapse-hide
income
## LONG FORMATED FINANCIALS (Year in columns)


,Year,04/30/2020,12/31/2019,12/31/2018,12/31/2017,12/31/2016
0,Total Revenue,8781000,8781000,8264000,7933000,5931000
1,Cost of Revenue,6802000,6802000,6591000,5633000,3875000
2,Gross Profit,1979000,1979000,1673000,2300000,2056000
3,Selling General and Administrative,313000,313000,326000,357000,225000
4,Total Operating Expenses,872000,872000,898000,922000,590000
5,Operating Income or Loss,1107000,1107000,775000,1378000,1466000
6,Interest Expense,63000,63000,73000,86000,30000
7,Total Other Income/Expenses Net,-70000,-70000,-155000,-119000,-118000
8,Income Before Tax,1016000,1016000,585000,1207000,1345000
9,Income Tax Expense,247000,247000,148000,173000,531000


In [0]:
#hide
## WIDE FORMATTED FINANCIALS (Metric In Columns)

def formatStmnt(inputs):
  ColumnList = list(inputs.columns)
  yearList = list(inputs.Year)

  print(ColumnList)
  print(yearList)


  transposed = inputs.transpose()
  cleaned = transposed.reset_index()

  new_header = cleaned.iloc[0] #grab the first row for the header
  cleaned = cleaned[1:] #take the data less the header row
  cleaned.columns = new_header #set the header row as the df header

  byYear = cleaned.set_index("Year").stack()

  df = pd.DataFrame(byYear, columns=['Value'])
  df.reset_index(inplace=True)
  print(df.columns)
  df.rename(columns={0:'Metric'}, inplace=True)
  df.dropna(inplace=True)
  df['Value'] = pd.to_numeric(df['Value'], errors='coerce')

  return df

In [312]:
#hide
#stack the transposed dataframes.
df = formatStmnt(income)


['Year', '04/30/2020', '12/31/2019', '12/31/2018', '12/31/2017', '12/31/2016']
['Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Selling General and Administrative', 'Total Operating Expenses', 'Operating Income or Loss', 'Interest Expense', 'Total Other Income/Expenses Net', 'Income Before Tax', 'Income Tax Expense', 'Income from Continuing Operations', 'Net Income', 'Net Income available to common shareholders', 'Basic EPS', 'Diluted EPS', 'Basic Average Shares', 'Diluted Average Shares']
Index(['Year', 0, 'Value'], dtype='object')


In [0]:
#hide
####PICK METRICS
columnList = ['Total Revenue',
              'Cost of Revenue', 
              'Gross Profit', 
              'Research Development', 
              'Selling General and Administrative', 
              'Total Operating Expenses', 
              'Operating Income or Loss', 
              'Interest Expense',
              'Total Other Income/Expenses Net', 
              'Income Before Tax',
              'Income Tax Expense',
              'Income from Continuing Operations',
              'Net Income', 
              'Net Income available to common shareholders', 
              'Basic EPS', 
              'Diluted EPS', 
              'Basic Average Shares',
              'Diluted Average Shares']

revenueBases = ['Total Revenue','Gross Profit', 'Operating Income or Loss','Income Before Tax', 'Net Income', 'Net Income available to common shareholders']
epsS = ['Basic EPS','Diluted EPS']
costs = [  'Cost of Revenue','Total Operating Expenses','Interest Expense', 'Income Tax Expense', 'Net Income', ]

oppCosts = ['Research Development', 'Selling General and Administrative']

floats = ['Basic Average Shares','Diluted Average Shares']


revs_g = df[df['Metric'].isin(revenueBases)]
eps_g = df[df['Metric'].isin(epsS)]
cost_g = df[df['Metric'].isin(costs)]
operating_g = df[df['Metric'].isin(oppCosts)]
float_g = df[df['Metric'].isin(floats)]


In [0]:
#hide
most_recent_float = list((float_g.Value).dropna())[0]

### Revenues

In [315]:
#collapse-hide

make_multi_LineGraph(revs_g, True)


alt.LayerChart(...)

### EPS

In [316]:
#collapse-hide

make_multi_LineGraph(eps_g, False)



alt.LayerChart(...)

### Cost Components

In [317]:
#collapse-hide

make_multi_LineGraph(cost_g, False)


alt.LayerChart(...)

In [318]:
#collapse-hide

make_multi_LineGraph(operating_g, False)


alt.LayerChart(...)

### Shares Outstanding

In [319]:
#collapse-hide

make_multi_LineGraph(float_g, False)

alt.LayerChart(...)

## Balance Sheet

In [320]:
#collapse-hide

balance

,Year,12/31/2019,12/31/2018,12/31/2017,12/31/2016
0,Current Assets,None,None,None,None
1,Cash,None,None,None,None
2,Cash And Cash Equivalents,221000,105000,194000,328000
3,Other Short Term Investments,1300000,1131000,1427000,1252000
4,Total Cash,1521000,1236000,1621000,1580000
5,Net Receivables,323000,366000,341000,302000
6,Inventory,72000,60000,57000,47000
7,Other Current Assets,516000,551000,2019000,470000
8,Total Current Assets,2037000,1787000,2146000,2050000
9,Non-current assets,None,None,None,None


In [321]:
#hide
df = formatStmnt(balance)

['Year', '12/31/2019', '12/31/2018', '12/31/2017', '12/31/2016']
['Current Assets', 'Cash', 'Cash And Cash Equivalents', 'Other Short Term Investments', 'Total Cash', 'Net Receivables', 'Inventory', 'Other Current Assets', 'Total Current Assets', 'Non-current assets', 'Property plant and equipment', 'Gross property plant and equipment', 'Accumulated Depreciation', 'Net property plant and equipment', 'Goodwill', 'Intangible Assets', 'Other long-term assets', 'Total non-current assets', 'Total Assets', 'Liabilities', 'Current Liabilities', 'Current Debt', 'Accounts Payable', 'Accrued liabilities', 'Deferred revenues', 'Other Current Liabilities', 'Total Current Liabilities', 'Non-current liabilities', 'Long Term Debt', 'Deferred taxes liabilities', 'Deferred revenues', 'Other long-term liabilities', 'Total non-current liabilities', 'Total Liabilities', "Stockholders' Equity", 'Common Stock', 'Retained Earnings', 'Accumulated other comprehensive income', "Total stockholders' equity", "Tot

In [0]:
#hide
columnList = ['Current Assets',
              'Cash',
              'Cash And Cash Equivalents',
              'Other Short Term Investments',
              'Total Cash',
              'Net Receivables',
              'Inventory',
              'Total Current Assets',
              'Non-current assets',
              'Property plant and equipment',
              'Gross property plant and equipment',
              'Accumulated Depreciation',
              'Net property plant and equipment',
              'Goodwill',
              'Intangible Assets',
              'Other long-term assets',
              'Total non-current assets',
              'Total Assets',
              'Liabilities',
              'Current Liabilities',
              'Current Debt',
              'Accounts Payable',
              'Accrued liabilities',
              'Deferred revenues',
              'Other Current Liabilities',
              'Total Current Liabilities',
              'Non-current liabilities',
              'Long Term Debt',
              'Deferred taxes liabilities',
              'Deferred revenues',
              'Other long-term liabilities',
              'Total non-current liabilities',
              'Total Liabilities',
              "Stockholders' Equity",
              'Common Stock',
              'Retained Earnings',
              'Accumulated other comprehensive income',
              "Total stockholders' equity",
              "Total liabilities and stockholders' equity"]

assets = ['Current Assets',
              'Cash',
              'Cash And Cash Equivalents',
              'Other Short Term Investments',
              'Total Cash',
              'Net Receivables',
              'Inventory',
              'Total Current Assets',
              'Non-current assets',
             
              'Goodwill',
              'Intangible Assets',
              'Other long-term assets',
              'Total non-current assets']

depre = [      'Property plant and equipment',
              'Gross property plant and equipment',
              'Accumulated Depreciation',
               'Net property plant and equipment']

current_liabs = [
              'Liabilities',
              'Current Liabilities',
              'Current Debt',
              'Accounts Payable',
              'Accrued liabilities',
              'Deferred revenues',
              'Other Current Liabilities'      


]

long_liabs = ['Non-current liabilities',
              'Deferred taxes liabilities',
              'Deferred revenues',
              'Other long-term liabilities',]


long_short = ['Total Liabilities','Total non-current liabilities', 'Total Current Liabilities', 'Long Term Debt']

liab_equi = [              'Total Liabilities',
              "Stockholders' Equity",]


equity = [ 'Common Stock',
              'Retained Earnings']

la = [        'Total Liabilities',
              "Total stockholders' equity",
              "Total liabilities and stockholders' equity"]           

assets_g = df[df['Metric'].isin(assets)]
dep_g = df[df['Metric'].isin(depre)]
clab_g = df[df['Metric'].isin(current_liabs)]
llab_g = df[df['Metric'].isin(long_liabs)]
ls_g = df[df['Metric'].isin(long_short)]
e =  df[df['Metric'].isin(equity)]
le = df[df['Metric'].isin(liab_equi)]

liabs_ass = df[df['Metric'].isin(la)]


### Assets

In [323]:
#collapse-hide

make_multi_LineGraph(assets_g, False)


alt.LayerChart(...)

### Depreceation

In [324]:
#collapse-hide

make_multi_LineGraph(dep_g, False)


alt.LayerChart(...)

### Current Liabilities

In [325]:
#collapse-hide

make_multi_LineGraph(clab_g, False)

alt.LayerChart(...)

### Long Term Liabilitys

In [326]:
#collapse-hide

make_multi_LineGraph(llab_g, False)


alt.LayerChart(...)

### Long and Short Term Liabilitys

In [327]:
#collapse-hide

make_multi_LineGraph(ls_g, False)


alt.LayerChart(...)

### Liabilitys and Equity

In [328]:
#collapse-hide

make_multi_LineGraph(le, False)


alt.LayerChart(...)

### Equity

In [329]:
#collapse-hide

make_multi_LineGraph(e, True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


alt.LayerChart(...)

### Total Assets & Total Liabilitys

In [330]:
#collapse-hide

make_multi_LineGraph(liabs_ass, False)

alt.LayerChart(...)

## Cash Flows

In [331]:
#collapse-hide

cash

,Year,04/30/2020,12/31/2019,12/31/2018,12/31/2017,12/31/2016
0,Net Income,769000,769000,437000,1034000,814000
1,Depreciation & amortization,423000,423000,398000,372000,363000
2,Stock based compensation,29000,29000,47000,55000,26000
3,Change in working capital,501000,501000,313000,129000,183000
4,Accounts receivable,43000,43000,-25000,-39000,-46000
5,Other working capital,1026000,1026000,235000,564000,708000
6,Net cash provided by operating activites,1722000,1722000,1195000,1590000,1386000
7,Investments in property plant and equipment,-696000,-696000,-960000,-1026000,-678000
8,Acquisitions net,-,-,0,0,-1951000
9,Purchases of investments,-1810000,-1810000,-834000,-1569000,-960000


# Peer Comparison

In [332]:
#hide
import sys
import requests
import bs4
res = requests.get('https://en.wikipedia.org/wiki/Russell_1000_Index')
# res = requests.get("http://www.nationmaster.com/country-info/stats/Media/Internet-users")
soup = BeautifulSoup(res.content,'lxml')
a1 = soup.find_all('table')
getReturns = a1[2]
getTickers = a1[3]
# print(a1)



df1 = pd.read_html(str(getReturns))
AnnualReturns = df1[0]
# a1 = soup.find_all('table')[3]
df2 = pd.read_html(str(getTickers))
print(len(df2))
russ1kList = df2[0]



1


In [0]:
#hide
IWB_comps = list(russ1kList['Ticker'])

In [0]:
#hide
import random
randomSample = []

# simuls = 30


# for i in range(simuls):
#   secure_random = random.SystemRandom()
#   a = secure_random.choice(IWB_comps)
#   randomSample.append(a)
#   # print(a)
# for i in tickerList:
#   randomSample.append(i)

In [0]:
#hide
yfStatsComp = yf_summaryTable(randomSample)
sumTable = pd.DataFrame.from_records(yfStatsComp)

In [336]:
#hide
new = sumTable.transpose()
new.columns

Index([], dtype='object')

In [0]:
#hide
vects= ['Index', 
        'P/E', 
        'EPS (ttm)', 
        'Insider Own',
        'Shs Outstand',
        'Perf Week',
        'Market Cap', 
        'Forward P/E',
        'EPS next Y',
        'Insider Trans',
        'Shs Float', 
        'Perf Month',
        'Income',
        'PEG',
        'EPS next Q',
        'Inst Own', 
        'Short Float',
        'Perf Quarter',
        'Sales',
        'P/S',
        'EPS this Y',
        'Inst Trans',
        'Short Ratio',
        'Perf Half Y',
        'Book/sh',
        'P/B',
        'ROA',
        'Target Price',
        'Perf Year',
        'Cash/sh',
        'P/C',
        'EPS next 5Y',
        'ROE',
        '52W Range',
        'Perf YTD', 
        'Dividend', 
        'P/FCF',
        'EPS past 5Y', 
        'ROI',
        '52W High',
        'Beta', 
        'Dividend %',
        'Quick Ratio',
        'Sales past 5Y',
        'Gross Margin',
        '52W Low',
        'ATR',
        'Employees',
        'Current Ratio',
        'Sales Q/Q',
        'Oper. Margin',
        'RSI (14)',
        'Volatility',
        'Optionable',
        'Debt/Eq',
        'EPS Q/Q', 
        'Profit Margin',
        'Rel Volume',
        'Prev Close',
        'Shortable',
        'LT Debt/Eq',
        'Earnings',
        'Payout',
        'Avg Volume',
        'Price',
        'Recom', 
        'SMA20',
        'SMA50',
        'SMA200',
        'Volume',
        'Change']

In [338]:
#hide
!pip install finviz
import finviz
recs = []
xyz= 0
for i in tickerList:
    try:
      hold = [i]
      for b in (finviz.get_stock(i)):
        colls = (list((finviz.get_stock(i)).keys()))
        hold.append(finviz.get_stock(i)[b])
      recs.append(hold)
    except BaseException as e:
      print(e, "could not find", i)
print(colls)


['Company', 'Sector', 'Industry', 'Country', 'Index', 'P/E', 'EPS (ttm)', 'Insider Own', 'Shs Outstand', 'Perf Week', 'Market Cap', 'Forward P/E', 'EPS next Y', 'Insider Trans', 'Shs Float', 'Perf Month', 'Income', 'PEG', 'EPS next Q', 'Inst Own', 'Short Float', 'Perf Quarter', 'Sales', 'P/S', 'EPS this Y', 'Inst Trans', 'Short Ratio', 'Perf Half Y', 'Book/sh', 'P/B', 'ROA', 'Target Price', 'Perf Year', 'Cash/sh', 'P/C', 'EPS next 5Y', 'ROE', '52W Range', 'Perf YTD', 'Dividend', 'P/FCF', 'EPS past 5Y', 'ROI', '52W High', 'Beta', 'Dividend %', 'Quick Ratio', 'Sales past 5Y', 'Gross Margin', '52W Low', 'ATR', 'Employees', 'Current Ratio', 'Sales Q/Q', 'Oper. Margin', 'RSI (14)', 'Volatility', 'Optionable', 'Debt/Eq', 'EPS Q/Q', 'Profit Margin', 'Rel Volume', 'Prev Close', 'Shortable', 'LT Debt/Eq', 'Earnings', 'Payout', 'Avg Volume', 'Price', 'Recom', 'SMA20', 'SMA50', 'SMA200', 'Volume', 'Change']


In [339]:
#hide
listCols = (list(['Ticker']))
for i in colls:
  # print(i)
  listCols.append(i)

print(listCols)
bigTable = pd.DataFrame.from_records(recs, columns=listCols)


['Ticker', 'Company', 'Sector', 'Industry', 'Country', 'Index', 'P/E', 'EPS (ttm)', 'Insider Own', 'Shs Outstand', 'Perf Week', 'Market Cap', 'Forward P/E', 'EPS next Y', 'Insider Trans', 'Shs Float', 'Perf Month', 'Income', 'PEG', 'EPS next Q', 'Inst Own', 'Short Float', 'Perf Quarter', 'Sales', 'P/S', 'EPS this Y', 'Inst Trans', 'Short Ratio', 'Perf Half Y', 'Book/sh', 'P/B', 'ROA', 'Target Price', 'Perf Year', 'Cash/sh', 'P/C', 'EPS next 5Y', 'ROE', '52W Range', 'Perf YTD', 'Dividend', 'P/FCF', 'EPS past 5Y', 'ROI', '52W High', 'Beta', 'Dividend %', 'Quick Ratio', 'Sales past 5Y', 'Gross Margin', '52W Low', 'ATR', 'Employees', 'Current Ratio', 'Sales Q/Q', 'Oper. Margin', 'RSI (14)', 'Volatility', 'Optionable', 'Debt/Eq', 'EPS Q/Q', 'Profit Margin', 'Rel Volume', 'Prev Close', 'Shortable', 'LT Debt/Eq', 'Earnings', 'Payout', 'Avg Volume', 'Price', 'Recom', 'SMA20', 'SMA50', 'SMA200', 'Volume', 'Change']


## Big Comp Table

In [340]:
#collapse-hide

bigTable

,Ticker,Company,Sector,Industry,Country,Index,P/E,EPS (ttm),Insider Own,Shs Outstand,Perf Week,Market Cap,Forward P/E,EPS next Y,Insider Trans,Shs Float,Perf Month,Income,PEG,EPS next Q,Inst Own,Short Float,Perf Quarter,Sales,P/S,EPS this Y,Inst Trans,Short Ratio,Perf Half Y,Book/sh,P/B,ROA,Target Price,Perf Year,Cash/sh,P/C,EPS next 5Y,ROE,52W Range,Perf YTD,Dividend,P/FCF,EPS past 5Y,ROI,52W High,Beta,Dividend %,Quick Ratio,Sales past 5Y,Gross Margin,52W Low,ATR,Employees,Current Ratio,Sales Q/Q,Oper. Margin,RSI (14),Volatility,Optionable,Debt/Eq,EPS Q/Q,Profit Margin,Rel Volume,Prev Close,Shortable,LT Debt/Eq,Earnings,Payout,Avg Volume,Price,Recom,SMA20,SMA50,SMA200,Volume,Change
0,ALK,"Alaska Air Group, Inc.",Services,Regional Airlines,USA,S&P 500,5.50,6.18,0.30%,123.13M,23.50%,4.19B,8.20,216.00%,-1.80%,122.05M,20.01%,769.00M,4.44,-1.08,90.00%,3.27%,-49.17%,8.78B,0.48,75.50%,-0.59%,1.40,-51.48%,35.18,0.97,6.00%,45.25,-45.07%,12.35,2.75,1.24%,18.90%,20.02 - 72.22,-49.82%,-,3.02,6.90%,14.00%,-52.92%,1.42,-,0.60,10.30%,56.10%,69.83%,2.92,24134,0.60,7.90%,12.10%,54.94,6.89% 8.15%,Yes,0.35,683.70%,8.80%,1.40,31.58,Yes,0.29,May 05 BMO,22.40%,2.85M,34.00,2.30,19.27%,-9.09%,-42.09%,"3,985,866",7.66%
1,AAL,American Airlines Group Inc.,Services,Major Airlines,USA,S&P 500,3.32,3.81,1.30%,426.89M,22.98%,5.39B,11.03,108.50%,0.98%,421.02M,3.10%,1.69B,0.29,-2.26,84.30%,19.37%,-52.87%,45.77B,0.12,25.20%,-0.88%,1.64,-59.73%,-0.27,-,2.80%,16.25,-63.05%,8.96,1.41,11.28%,-,9.09 - 34.99,-55.96%,0.40,-,-0.70%,10.30%,-63.90%,1.81,3.17%,0.30,1.40%,61.50%,38.94%,1.36,133000,0.40,3.40%,6.70%,51.95,7.34% 9.51%,Yes,-,35.10%,3.70%,2.21,11.26,Yes,-,Apr 30 BMO,10.50%,49.85M,12.63,3.20,15.61%,-15.87%,-49.70%,"110,358,587",12.17%
2,DAL,"Delta Air Lines, Inc.",Services,Major Airlines,USA,S&P 500,5.07,5.39,0.30%,639.81M,21.58%,17.48B,6.61,170.10%,-86.23%,639.81M,-4.71%,3.50B,1.32,-4.17,88.70%,3.91%,-51.77%,45.13B,0.39,28.70%,-0.15%,0.71,-50.48%,22.46,1.22,5.40%,38.93,-53.13%,-,-,3.85%,23.90%,19.10 - 63.44,-53.28%,-,4.14,56.40%,19.60%,-56.93%,1.33,-,0.50,3.10%,53.50%,43.04%,2.54,91224,0.50,-18.00%,11.50%,50.93,6.29% 8.27%,Yes,1.19,-176.60%,7.80%,2.27,24.34,Yes,0.88,Apr 22 BMO,28.80%,35.19M,27.32,2.20,16.37%,-19.29%,-47.26%,"80,009,047",12.24%
3,LUV,Southwest Airlines Co.,Services,Regional Airlines,USA,S&P 500,7.34,4.22,0.30%,526.00M,4.24%,16.29B,9.87,190.80%,-59.39%,506.62M,-11.77%,2.27B,4.59,-0.41,89.10%,1.91%,-45.56%,22.43B,0.73,-2.00%,0.57%,0.95,-46.27%,18.69,1.66,8.70%,43.06,-42.89%,7.74,4.00,1.60%,23.30%,28.25 - 58.83,-42.63%,0.72,6.29,20.80%,18.20%,-47.36%,1.40,2.32%,0.60,3.80%,74.50%,9.63%,2.56,60800,0.70,0.50%,13.20%,42.21,5.85% 6.69%,Yes,0.27,-21.80%,10.30%,9.86,29.69,Yes,0.19,Apr 28 BMO,16.40%,10.18M,30.97,2.60,-2.73%,-20.46%,-38.70%,"100,371,268",4.31%
4,UAL,"United Airlines Holdings, Inc.",Services,Major Airlines,USA,S&P 500,2.68,11.63,0.30%,248.79M,20.60%,7.76B,5.74,134.70%,-0.27%,246.21M,3.48%,3.01B,0.26,-2.88,-,9.11%,-58.50%,43.26B,0.18,51.30%,0.22%,1.06,-66.15%,45.72,0.68,5.80%,49.50,-64.88%,19.87,1.57,10.20%,28.00%,17.80 - 96.03,-64.57%,-,3.26,31.60%,12.90%,-67.50%,1.56,-,0.50,2.10%,62.00%,75.34%,3.49,96000,0.50,3.80%,9.90%,48.35,6.98% 10.68%,Yes,1.29,51.20%,7.00%,2.44,27.84,Yes,1.16,Apr 30 AMC,0.00%,21.26M,31.21,2.40,14.40%,-23.51%,-58.95%,"51,784,724",12.10%


In [0]:
#hide
# import numpy as np 
# from pandas import DataFrame
# import matplotlib.pyplot as plt


# plt.pcolor(bigTable)
# plt.yticks(np.arange(0.5, len(df.index), 1), bigTable.index)
# plt.xticks(np.arange(0.5, len(df.columns), 1), bigTable.columns)
# plt.show()

In [0]:
  # hide
  # new_header = df.iloc[0] #grab the first row for the header
  # df = df[1:] #take the data less the header row
  # df.columns = new_header #set the header row as the df header 

In [0]:
#hide
#PE data http://www.stern.nyu.edu/~adamodar/pc/datasets/pedata.xls
#
#
#
#

Market Value

Enterprise Value

Leverage

# DCF Valuation

In [344]:
#hide
!pip install pyfinmod

In [0]:
#hide


from pyfinmod.financials import Financials
from pyfinmod.ev import fcf, enterprise_value

from pyfinmod.financials import Financials

from pyfinmod.financials import Financials
from pyfinmod.wacc import total_debt

from pyfinmod.financials import Financials
from pyfinmod.wacc import tax_rate





from pyfinmod.financials import Financials
from pyfinmod.wacc import cost_of_debt

from pyfinmod.financials import Financials
from pyfinmod.wacc import cost_of_equity

# RFR = .01
# mkt_return = .09
# LR_g =.1
# SR_g = .1 + LR_g

# print((RFR,mkt_return,SR_g,LR_g))




# lec = (new.loc['Weighted Average Shs Out'][0])

# eq = parser.balance_sheet_statement.loc['Total liabilities'][0]




# Calculate WACC


parser = Financials(subjectCompany)
from pyfinmod.wacc import wacc

from pyfinmod.ev import dcf, fcf
from pyfinmod.financials import Financials

def fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g, shock, subjectCompany=subjectCompany):
  # print(fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g))
  parser = Financials(subjectCompany)
  lec = (parser.balance_sheet_statement.iloc[-5::5])

  for i in lec.itertuples():
    ec = (i._1)
    break


  parser = Financials(subjectCompany)
  parser.income_statement.columns
  currentPrice = parser.price


  parser = Financials(subjectCompany)
  ev = enterprise_value(parser.balance_sheet_statement)[0]
  ecToev = ec / ev
  equitys_share = (1-ecToev)






  parser = Financials(subjectCompany)
  fcf_value = fcf(parser.cash_flow_statement)
  total_debt(parser.balance_sheet_statement)
  tax_rate(parser.income_statement)
  parser.mktCap



  # cost_of_debt(parser.balance_sheet_statement, parser.income_statement)

  # cost_of_equity(float(parser.beta), risk_free_interest_rate = RFR, market_return = mkt_return)
  parser = Financials(subjectCompany)

  aapl_wacc = wacc(parser.mktCap,
                  parser.balance_sheet_statement,
                  parser.income_statement,
                  float(parser.beta),
                  risk_free_interest_rate=RFR,
                  market_return=mkt_return)
  

  parser = Financials(subjectCompany)
  if shock:
    output = dcf_n1Shock(fcf(parser.cash_flow_statement), aapl_wacc, shock, short_term_growth=SR_g, long_term_growth=LR_g)
  else:
    output = dcf(fcf(parser.cash_flow_statement), aapl_wacc, short_term_growth=SR_g, long_term_growth=LR_g)
  
  value_per_share = output / (parser.income_statement.loc['Weighted Average Shs Out'][0])

  pps = equitys_share * value_per_share

  IR = ((pps - currentPrice) / currentPrice)*100

  out = (value_per_share, pps, SR_g, LR_g, IR)
  return (out)
  # return float(value_per_share)


Generating a range of growth rate scenarios:

In [0]:
#collapse-hide

SR_g_scenarios = np.linspace(-.03,.04,15,endpoint=False)





### DCF Growth Rate Scenarios

In [347]:
#hide
parser = Financials(subjectCompany)
parser.income_statement.columns

currentPrice = parser.price
colz = ('Firm Value/Share ($)','Equity Value/Share ($)', 'SR CAGR', 'LR CAGR', 'Implied ROI (%)')

records_dcf_1 = [colz]
print("Current Price:", currentPrice)
RFR = .02
mkt_return = .12
for i in range(len(SR_g_scenarios)):
  SR_g = SR_g_scenarios[i]
  LR_g = .01
  records_dcf_1.append(fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g, False))
  # print(records_dcf_1)
  print("Done computing ", (i + 1), ' DCF out of ',len(SR_g_scenarios))

dcf_growth_rates_outputs = pd.DataFrame.from_records(records_dcf_1, columns = list(records_dcf_1[0]))



Current Price: 34.0
Done computing  1  DCF out of  15
Done computing  2  DCF out of  15
Done computing  3  DCF out of  15
Done computing  4  DCF out of  15
Done computing  5  DCF out of  15
Done computing  6  DCF out of  15
Done computing  7  DCF out of  15
Done computing  8  DCF out of  15
Done computing  9  DCF out of  15
Done computing  10  DCF out of  15
Done computing  11  DCF out of  15
Done computing  12  DCF out of  15
Done computing  13  DCF out of  15
Done computing  14  DCF out of  15
Done computing  15  DCF out of  15


In [348]:
#collapse-hide

ind = (dcf_growth_rates_outputs.columns[4])
dcf_growth_rates_outputs


,Firm Value/Share ($),Equity Value/Share ($),SR CAGR,LR CAGR,Implied ROI (%)
0,Firm Value/Share ($),Equity Value/Share ($),SR CAGR,LR CAGR,Implied ROI (%)
1,138.905,77.4672,-0.03,0.01,127.845
2,141.907,79.1414,-0.0253333,0.01,132.769
3,144.963,80.846,-0.0206667,0.01,137.782
4,148.075,82.5815,-0.016,0.01,142.887
5,151.243,84.3483,-0.0113333,0.01,148.083
6,154.468,86.147,-0.00666667,0.01,153.373
7,157.751,87.9778,-0.002,0.01,158.758
8,161.092,89.8412,0.00266667,0.01,164.239
9,164.493,91.7378,0.00733333,0.01,169.817


Seems to do a decent job, however this does not take into account the shock from coronavirus which will probably hurt revenues next year. We will modifiy the discounted cash flow formula to include a shock to FCF in the first year.

In [349]:
#hide
from pyfinmod.basic import npv
from math import sqrt

def dcf_n1Shock(fcfs, wacc, SHOCK, short_term_growth, long_term_growth):
    latest_fcf_date = fcfs.index.max()
    dates = pd.date_range(latest_fcf_date, periods=6, freq="365D")[1:]
    future_cash_flows = [fcfs[latest_fcf_date]]
    for i in range(5): # 5?
        # print(i)
        # print(SHOCK)
        if i == 0:        
          next_year_fcf = future_cash_flows[-1] * (1 + short_term_growth)
          future_cash_flows.append(next_year_fcf*SHOCK)

        else:
            
          next_year_fcf = future_cash_flows[-1] * (1 + short_term_growth)
          future_cash_flows.append(next_year_fcf)



    future_cash_flows = future_cash_flows[1:]
    df = pd.DataFrame(data={"fcf": future_cash_flows, "date": dates})
    # df.set_index('date', inplace=True)
    df["terminal value"] = 0
    last_index = df.index[-1]
    last_short_term_fcf = df.at[last_index, "fcf"]
    df.at[last_index, "terminal value"] = (
        last_short_term_fcf * (1 + long_term_growth) / (wacc - long_term_growth)
    )
    df["cash flow"] = df[["fcf", "terminal value"]].apply(sum, axis=1)
    return npv(df, wacc) * sqrt((1 + wacc))

parser = Financials(subjectCompany)
parser.income_statement.columns

currentPrice = parser.price
colz = ('Firm Value/Share ($)','Equity Value/Share ($)', 'SR CAGR', 'LR CAGR', 'Implied ROI (%)')

records_dcf_1 = [colz]
print("Current Price:", currentPrice)
RFR = .02
mkt_return = .12
for i in range(len(SR_g_scenarios)):
  SR_g = SR_g_scenarios[i]
  LR_g = .01
  records_dcf_1.append(fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g, 1/2))
  # print(records_dcf_1)
  print("Done computing ", (i + 1), ' DCF out of ',len(SR_g_scenarios))

dcf_growth_rates_outputs_shock = pd.DataFrame.from_records(records_dcf_1, columns = list(records_dcf_1[0]))



Current Price: 34.0
Done computing  1  DCF out of  15
Done computing  2  DCF out of  15
Done computing  3  DCF out of  15
Done computing  4  DCF out of  15
Done computing  5  DCF out of  15
Done computing  6  DCF out of  15
Done computing  7  DCF out of  15
Done computing  8  DCF out of  15
Done computing  9  DCF out of  15
Done computing  10  DCF out of  15
Done computing  11  DCF out of  15
Done computing  12  DCF out of  15
Done computing  13  DCF out of  15
Done computing  14  DCF out of  15
Done computing  15  DCF out of  15


In [350]:
dcf_growth_rates_outputs_shock

,Firm Value/Share ($),Equity Value/Share ($),SR CAGR,LR CAGR,Implied ROI (%)
0,Firm Value/Share ($),Equity Value/Share ($),SR CAGR,LR CAGR,Implied ROI (%)
1,69.4524,38.7336,-0.03,0.01,13.9224
2,70.9533,39.5707,-0.0253333,0.01,16.3844
3,72.4816,40.423,-0.0206667,0.01,18.8911
4,74.0375,41.2907,-0.016,0.01,21.4434
5,75.6216,42.1742,-0.0113333,0.01,24.0417
6,77.2341,43.0735,-0.00666667,0.01,26.6867
7,78.8755,43.9889,-0.002,0.01,29.3791
8,80.5462,44.9206,0.00266667,0.01,32.1195
9,82.2465,45.8689,0.00733333,0.01,34.9085


## DCF Peer Comparison

In [351]:

parser = Financials(subjectCompany)
parser.income_statement.columns

currentPrice = parser.price
colz = ('Firm Value/Share ($)','Equity Value/Share ($)', 'SR CAGR', 'LR CAGR', 'Implied ROI (%)')

records_dcf_2 = []
print("Current Price:", currentPrice)
RFR = .02
mkt_return = .12
LR_g = .01
SR_g = .02
for i in tickerList:
  records_dcf_2.append(fcf_dcf_using_wacc(RFR,mkt_return,SR_g,LR_g, .5, i))
  # print(records_dcf_1)


peer_dcf = pd.DataFrame.from_records(records_dcf_2, columns = list(records_dcf_1[0]))

peer_dcf['ticker'] = tickerList
peer_dcf.set_index('ticker', inplace=True)

Current Price: 34.0


/usr/local/lib/python3.6/dist-packages/pyfinmod/wacc.py:9: RuntimeWarning: invalid value encountered in double_scalars
  return income_tax_expense / income_before_tax


In [352]:
peer_dcf

,Firm Value/Share ($),Equity Value/Share ($),SR CAGR,LR CAGR,Implied ROI (%)
ticker,,,,,
ALK,87.014531,48.528018,0.02,0.01,42.729465
AAL,54.319490,54.473259,0.02,0.01,331.300545
DAL,58.522847,38.246862,0.02,0.01,39.995833
LUV,28.131268,11.806731,0.02,0.01,-61.876877
UAL,670.864513,465.525445,0.02,0.01,1391.590660


## Historical DCF Scenario Backtest Analysis Using Differint Bases (Work In Progress.)

(WIP -- NEED TO DO MORE CODE BUT THIS WILL HOW HOW ACCURATE OUR MODELS ARE OVER TIME AND WHAT EARNINGS BASE MAKE THE BEST ESTIMATORS FOR THIS FIRM)

In [0]:
from google.colab import drive
import sys

In [354]:
!pip install matplotlib urllib3 seaborn
!git clone https://github.com/halessi/DCF.git
!ls DCF
!python DCF/main.py --t ALK --i 'quarter' --y 3 --eg .15 --steps 2 --s 0.1 --v eg

fatal: destination path 'DCF' already exists and is not an empty directory.
imgs  main.py  modeling  README.md  visualization
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Forecasting flows for 5 years out, starting at 2019-12-31. 
         DFCF   |    EBIT   |    D&A    |    CWC     |   CAP_EX   | 
2020   3.67E+08 |  2.94E+08 |  1.23E+08 |  -1.16E+08 |  1.77E+08 | 
2021   4.68E+08 |  3.90E+08 |  1.64E+08 |  -8.08E+07 |  1.92E+08 | 
2022   6.33E+08 |  5.84E+08 |  2.46E+08 |  -5.66E+07 |  2.18E+08 | 
2023   9.20E+08 |  9.69E+08 |  4.08E+08 |  -3.96E+07 |  2.58E+08 | 
2024   1.46E+09 |  1.77E+09 |  7.44E+08 |  -2.77E+07 |  3.16E+08 | 

Enterprise Value for ALK: $2.11E+10. 
Equity Value for ALK: $1.97E+10. 
Per share value for ALK: $1.59E+02.
 ------------------------------------------------------------
Forecasting f

# Conclusions and Summary

Why Now: 
The Coronavirus continues to impact the economy at an unprecedented level. The worst economic climate in the United States on record allows us to take advantage of our cash rich position. Under the presumption that the recession will eventually end, either by means of a vaccine or herd immunity, we decided to explore the wreckage that is the travel industry in search of pockets of value and mispriced companies. We have selected an airline with the most adaptable business model, further bolstered by government guarantees to bailout airlines and low fuel prices. Right now no one considers the prospects of traveling, but, as when Covid-19 first began, and surely when solved, young people and business people will promptly take advantage of travel deals and take the lead in traveling. Air transportation is a necessary proponent of the world economy globalization. 

Why Alaska Airlines: 
Alaska Airlines would compliment our portfolio as an international, US based travel company, additionally owning Horizon Air and McGee air services. Unlike other airlines, Alaska Airlines already procures a significant portion of their revenue from cargo flights, which many airlines are now adapting to. The bush in Alaska, served primarily by Alaska Airlines, needs sustained air travel to deliver necessities regardless of economic climate due to the rough terrain.

Why Alaska now: 
(Models, incorporating worst case scenarios for economic slowdowns, predict not only Alaska Airlines’ survival as a company but it’s undervaluation. The revenue they bring in through Alaska could sustain the company through the Coronavirus while maintaining positive outlooks for the future. As a company well versed in cargo shipping and dominates the Pacific Northwest, with areas that already rely on bush cargo flights, exposure is lower than other airlines. Alaska Airlines trades at cheap prices compared to fundamental value, and presents a significant opportunity to add a large airline to our portfolio, and take advantage of the recession and extreme pessimism.

Brief Industry Overview

Airline’s main sources of revenue include commercial and general flights. Commercial flights allow for cargo shipments and public leisure travel and general flights include private flights and lessons. Airline profits have traditionally depended on economic conditions. Fuel is the largest industry expense of airlines. 

Brief Peer Overview

Southwest,  American, United, Hawaiian, Delta and Air Canada Airlines act as Alaska Airlines’ main competitors operating in similar geographic locations. Alaska Airlines’ smaller size and dominance in a relatively small geographic location may benefit the company in a time of economic downturns. They have less debt and less planes, and thus lower operating costs.

Business Description

Commercial transportation and cargo flights drive Alaskan Airline revenue. Alaska has a history of succeeding in leisure travel in the Northamerican west,  Alaska Airlines provides the main source of connection to the continental United States to Alaska for cargo, business and leisure travel. If they were to go out of business, Alaska would be stranded. The United States Government could not reasonably go without bailing out Alaska Airlines if need be without jeopardizing the health and stability of Alaska. 

Sustainability and Governance
Alaska Airlines rates as the most environmentally sustainable airline in the United States. 

Risks
Alaska Airlines has many competitors, some much larger than itself which may have more leverage to sustain themselves. They also recently acquired Virgin Airlines which comes with rebranding and retraining expenses on top of the expense of purchasing an airline. Their survival depends on their speed of adaptation over other competitors and maintaining their dominance of the Pacific Northwest.  

Recommendation:

BUY: 
Using a compound annual growth rate of 0.0166667 (1.6%), we find ALK to be undervalued by around 40.6353% assuming an extreme shock to free cash flows resulting in 50% in the first year resulting from coronavirus.
